In [146]:
#markdown 8,9,10,11节代码实现
import pandas as pd
import numpy as np
#all-tags.csv是每个用户 对不同电影打的不同标签,三列分别是 user_id,movieid,tag
#里边第一列是零列，我们拿的1列，2列,是movieid,tag
_tags = pd.read_csv("./datasets/ml-latest-small/all-tags.csv", usecols=range(1, 3)).dropna()

C:\Users\HDS\AppData\Local\Temp\ipykernel_24560\2285456343.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  _tags = pd.read_csv("./datasets/ml-latest-small/all-tags.csv", usecols=range(1, 3)).dropna()


In [147]:
_tags.columns

Index(['movieId', 'tag'], dtype='object')

In [148]:
_tags.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1108987, 1108988, 1108989, 1108990, 1108991, 1108992, 1108993, 1108994,
       1108995, 1108996],
      dtype='int64', length=1108981)

In [149]:
_tags.count()

movieId    1108981
tag        1108981
dtype: int64

# 物品画像（电影画像）

In [150]:
# 步骤
# 1、找出所有的标签词，构建一本字典
# 2、通过字典把每部电影的标签词变为id
# 3、通过tfidf算出每个电影的词的权重
# 4、找出权重最大的词，进行排序，排到前面的30个，作为电影的画像
# 5、通过倒排索引技术来实现推荐(给了感兴趣的词，给出推荐的电影)

## 下面是UGC的内容

In [151]:
tags = _tags.groupby('movieId').agg(list)

In [152]:
tags.head()  #聚合后得到一部电影的所有标签

,tag
movieId,
1,"[animated, buddy movie, Cartoon, cgi, comedy, ..."
2,"[fantasy, adapted from:book, animals, bad cgi,..."
3,"[moldy, old, Ann Margaret, Burgess Meredith, D..."
4,"[characters, girl movie, characters, chick fli..."
5,"[steve martin, steve martin, pregnancy, remake..."


In [153]:
tags.shape

(56314, 1)

## 加载PGC的数据

In [154]:
 # 加载电影列表数据集，得到电影名字，电影题材
movies = pd.read_csv("./datasets/ml-latest-small/movies.csv", index_col="movieId")

In [155]:
movies.dtypes

title     object
genres    object
dtype: object

In [156]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [157]:
movies['genres'] = movies['genres'].apply(lambda x:x.split('|'))

In [158]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
5,Father of the Bride Part II (1995),[Comedy]


In [159]:
len(movies.index)

9742

In [160]:
len(tags.index)

56314

In [161]:
#拿到所有有标签的电影的索引，set去重求交集
movies_index = set(movies.index)&set(tags.index)


In [162]:
len(movies_index) #交集有9544部电影

9544

In [163]:
#把电影的题材，标签进行了结合，可以理解为PGC，UGC进行了结合
new_tags = tags.loc[list(movies_index)]  #通过movies_index先对用户评论进行过滤
ret = movies.join(new_tags)  #以专业影评为主，UGC没有的，就给nan,join是左连接

In [164]:
ret.shape

(9742, 3)

In [165]:
ret.iloc[0]

title                                      Toy Story (1995)
genres    [Adventure, Animation, Children, Comedy, Fantasy]
tag       [animated, buddy movie, Cartoon, cgi, comedy, ...
Name: 1, dtype: object

In [166]:
retll=list(ret.itertuples()) #感受itertuples功能

In [167]:
retll[0]  #里边有重复的tag

Pandas(Index=1, title='Toy Story (1995)', genres=['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], tag=['animated', 'buddy movie', 'Cartoon', 'cgi', 'comedy', 'computer animation', 'family', 'friendship', 'kids', 'toy', 'toys', 'adventure', 'animated', 'animation', 'buddy movie', 'children', 'classic', 'clever', 'comedy', 'computer animation', 'Disney', 'family', 'fantasy', 'funny', 'humorous', 'imdb top 250', 'Pixar', 'time travel', 'Tom Hanks', 'toys', 'witty', 'animation', 'cartoon', 'friendship', 'pixar', 'unny', 'adventure', 'animated', 'animation', 'computer animation', 'Disney', 'funny', 'pixar', 'Tom Hanks', 'toys', 'pixar', 'animation', 'children', 'Pixar', 'witty', 'animated', 'animation', 'children', 'comedy', 'fantasy', 'funny', 'humorous', 'Pixar', 'time travel', 'animation', 'Pixar', 'animation', 'fun', 'animation', 'Pixar', 'computer animation', 'funny', 'Pixar', 'animation', 'Pixar', 'toys', 'adventure', 'children', 'classic', 'computer animation', 'Disney', 

In [168]:
#如果x[3]不是空的，就加起来，否则就给个空，把genres加入了tags中，如果针对没有UGC的，让PGC内容代替，就是[]改为x[2]
temp = map(lambda x: (x[0],x[1],x[2],x[2]+x[3]) if x[3] is not np.nan else (x[0],x[1],x[2],x[2]),ret.itertuples())

In [169]:
temp

In [170]:
#tags中包含了genres，这时tags中包含了genres和tags，这样就可以把UGC和PGC结合起来了
movie_dataset = pd.DataFrame(temp,columns=["movieId", "title", "genres","tags"])

In [171]:
movie_dataset.head()

,movieId,title,genres,tags
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[Adventure, Animation, Children, Comedy, Fanta..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[Adventure, Children, Fantasy, fantasy, adapte..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[Comedy, Romance, moldy, old, Ann Margaret, Bu..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[Comedy, Drama, Romance, characters, girl movi..."
4,5,Father of the Bride Part II (1995),[Comedy],"[Comedy, steve martin, steve martin, pregnancy..."


In [ ]:
movie_dataset.set_index('movieId',inplace=True)  #把movieId设置为行索引

In [173]:
movie_dataset.head()

,title,genres,tags
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[Adventure, Animation, Children, Comedy, Fanta..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[Adventure, Children, Fantasy, fantasy, adapte..."
3,Grumpier Old Men (1995),"[Comedy, Romance]","[Comedy, Romance, moldy, old, Ann Margaret, Bu..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[Comedy, Drama, Romance, characters, girl movi..."
5,Father of the Bride Part II (1995),[Comedy],"[Comedy, steve martin, steve martin, pregnancy..."


In [174]:
#例如tags中animals有很多重复的
dataset =movie_dataset['tags'].values

In [175]:
dataset[0:5]

array([list(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'animated', 'buddy movie', 'Cartoon', 'cgi', 'comedy', 'computer animation', 'family', 'friendship', 'kids', 'toy', 'toys', 'adventure', 'animated', 'animation', 'buddy movie', 'children', 'classic', 'clever', 'comedy', 'computer animation', 'Disney', 'family', 'fantasy', 'funny', 'humorous', 'imdb top 250', 'Pixar', 'time travel', 'Tom Hanks', 'toys', 'witty', 'animation', 'cartoon', 'friendship', 'pixar', 'unny', 'adventure', 'animated', 'animation', 'computer animation', 'Disney', 'funny', 'pixar', 'Tom Hanks', 'toys', 'pixar', 'animation', 'children', 'Pixar', 'witty', 'animated', 'animation', 'children', 'comedy', 'fantasy', 'funny', 'humorous', 'Pixar', 'time travel', 'animation', 'Pixar', 'animation', 'fun', 'animation', 'Pixar', 'computer animation', 'funny', 'Pixar', 'animation', 'Pixar', 'toys', 'adventure', 'children', 'classic', 'computer animation', 'Disney', 'funny', 'Pixar', 'Tim Allen', 'Tom Hanks',

In [176]:
#需要安装,pip install gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

In [177]:
# 创建Dictionary对象，对词进行了去重
dct = Dictionary(dataset)

In [178]:
type(dct)

gensim.corpora.dictionary.Dictionary

In [179]:
#字典中有5万多个词,输出前10个
for key,value in dct.items():
    print(key,value)
    if 10==key:
        break

0 2009 reissue in Stereoscopic 3-D
1 3D
2 55 movies every kid should see--Entertainment Weekly
3 Adventure
4 American Animation
5 Animation
6 BD-Video
7 Best of Rotten Tomatoes: All Time
8 Buzz Lightyear
9 CG animation
10 CGI


In [180]:
#看下key=15的词是什么
dct[15]

'Comedy'

In [181]:
#通过单词的值去找对应的id，并统计键出现的次数，这个例子是为了学习doc2bow含义
dct.doc2bow(['Comedy', 'Romance', 'moldy', 'Comedy','old', 'Ann Margaret', 'Burgess Meredith', 'Daryl Hannah', 'fishing', 'good soundtrack','fishing', 'good soundtrack'])

[(15, 2),
 (166, 1),
 (167, 1),
 (168, 1),
 (173, 1),
 (180, 2),
 (181, 2),
 (183, 1),
 (184, 1)]

In [182]:
#把每一行数据转换为如上面看到的list，每一行就是一步电影
corpus = [dct.doc2bow(line) for line in dataset]

In [183]:
len(corpus)

9742

In [184]:
corpus[0] #看下第一个元素，一个元素对应一部电影

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 6),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 41),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 3),
 (24, 1),
 (25, 1),
 (26, 57),
 (27, 2),
 (28, 1),
 (29, 6),
 (30, 29),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 2),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 13),
 (41, 1),
 (42, 16),
 (43, 63),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 7),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 5),
 (53, 1),
 (54, 26),
 (55, 1),
 (56, 10),
 (57, 11),
 (58, 22),
 (59, 26),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 2),
 (65, 2),
 (66, 1),
 (67, 1),
 (68, 18),
 (69, 1),
 (70, 1),
 (71, 7),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 17),
 (77, 7),
 (78, 1),
 (79, 28),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 13),
 (85, 6),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 2),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99,

In [185]:
corpus[1]



[(3, 1),
 (12, 1),
 (14, 5),
 (21, 2),
 (40, 2),
 (53, 1),
 (54, 1),
 (58, 1),
 (68, 1),
 (71, 20),
 (88, 1),
 (91, 1),
 (114, 18),
 (130, 1),
 (131, 1),
 (132, 2),
 (133, 1),
 (134, 2),
 (135, 1),
 (136, 2),
 (137, 4),
 (138, 1),
 (139, 28),
 (140, 1),
 (141, 1),
 (142, 3),
 (143, 9),
 (144, 9),
 (145, 3),
 (146, 1),
 (147, 10),
 (148, 2),
 (149, 1),
 (150, 1),
 (151, 2),
 (152, 1),
 (153, 1),
 (154, 1),
 (155, 2),
 (156, 8),
 (157, 5),
 (158, 4),
 (159, 1),
 (160, 3),
 (161, 1),
 (162, 7),
 (163, 1),
 (164, 1),
 (165, 2)]

## 开始计算tfidf

In [186]:
# 训练TF-IDF模型，即计算TF-IDF值
model = TfidfModel(corpus)

In [187]:
#我们先来看下第一个的tf-idf值
vector = model[corpus[1]]
vector

[(3, 0.008925885902739789),
 (12, 0.009776392326012205),
 (14, 0.058647453849396604),
 (21, 0.021936617834063254),
 (40, 0.029945481689316062),
 (53, 0.019327944574581004),
 (54, 0.01721894136608525),
 (58, 0.010573988007313653),
 (68, 0.013466027617969459),
 (71, 0.2956929099413669),
 (88, 0.01666004284862406),
 (91, 0.021677760367632),
 (114, 0.3269139754351591),
 (130, 0.04022001910975634),
 (131, 0.025042656005685116),
 (132, 0.07081781766759998),
 (133, 0.023359216730608832),
 (134, 0.06634373990045275),
 (135, 0.03540890883379999),
 (136, 0.06222720249462721),
 (137, 0.1044948831627856),
 (138, 0.037184546488942094),
 (139, 0.6593901831527829),
 (140, 0.0275623259370294),
 (141, 0.031698370106942375),
 (142, 0.039605044837849476),
 (143, 0.19234254085350116),
 (144, 0.2383998357922951),
 (145, 0.031218879298976918),
 (146, 0.025473750806610653),
 (147, 0.2866289748612813),
 (148, 0.07081781766759998),
 (149, 0.025181691851438023),
 (150, 0.026887259830986027),
 (151, 0.0529777412

In [188]:
#按tfidf值进行排序，tfidf值越大越重要,取前30个作为电影的画像
movie_tags = sorted(vector, key=lambda x: x[1], reverse=True)[:30]

In [189]:
movie_tags

[(139, 0.6593901831527829),
 (114, 0.3269139754351591),
 (71, 0.2956929099413669),
 (147, 0.2866289748612813),
 (156, 0.24478238846274916),
 (144, 0.2383998357922951),
 (143, 0.19234254085350116),
 (157, 0.1859227324447105),
 (162, 0.14428813899752685),
 (137, 0.1044948831627856),
 (158, 0.0967053015376365),
 (160, 0.08423438587949808),
 (132, 0.07081781766759998),
 (148, 0.07081781766759998),
 (134, 0.06634373990045275),
 (136, 0.06222720249462721),
 (14, 0.058647453849396604),
 (151, 0.05297774128717669),
 (165, 0.047914633370299345),
 (155, 0.04429243846124757),
 (130, 0.04022001910975634),
 (163, 0.04022001910975634),
 (142, 0.039605044837849476),
 (138, 0.037184546488942094),
 (164, 0.037184546488942094),
 (135, 0.03540890883379999),
 (153, 0.03540890883379999),
 (152, 0.033171869950226376),
 (159, 0.033171869950226376),
 (141, 0.031698370106942375)]

In [190]:
#为每一部电影都筛选出来30个关键词，把id转换为词
dict(map(lambda x:(dct[x[0]], x[1]), movie_tags))

{'Robin Williams': 0.6593901831527829,
 'time travel': 0.3269139754351591,
 'fantasy': 0.2956929099413669,
 'board game': 0.2866289748612813,
 'kid flick': 0.24478238846274916,
 'bad cgi': 0.2383998357922951,
 'animals': 0.19234254085350116,
 'magic board game': 0.1859227324447105,
 'scary': 0.14428813899752685,
 'Kirsten Dunst': 0.1044948831627856,
 'monkey': 0.0967053015376365,
 'not for kids': 0.08423438587949808,
 'Chris Van Allsburg': 0.07081781766759998,
 'childhood recaptured': 0.07081781766759998,
 'Filmed in BC': 0.06634373990045275,
 'Joe Johnston': 0.06222720249462721,
 'Children': 0.058647453849396604,
 'game': 0.05297774128717669,
 'time': 0.047914633370299345,
 'jungle': 0.04429243846124757,
 'Adaptation of Book': 0.04022001910975634,
 'see also:Zathura': 0.04022001910975634,
 'adapted from:book': 0.039605044837849476,
 'Lebbat': 0.037184546488942094,
 'thrill': 0.037184546488942094,
 'For children': 0.03540890883379999,
 'herds of CGI animals': 0.03540890883379999,
 'gia

In [191]:
#对上面代码进行汇总,拿到所有电影的物品画像
movie_profile = {}
for i,mid in enumerate(movie_dataset.index):#电影id和原有ndarray中数据对应起来
    vector = model[corpus[i]] #计算某一步电影的tfidf
    movie_tags = sorted(vector,key = lambda x:x[1],reverse=True)[:30] #取前30个关键词
    movie_profile[mid] = dict(map(lambda x:(dct[x[0]],x[1]),movie_tags)) #把id转换为词，每部电影画像放入字典中

In [192]:
for k,v in movie_profile.items():
    print(k,v)
    if k>6:
        break

1 {'Pixar': 0.5233593756732888, 'pixar': 0.42629530966659107, 'animation': 0.36262507978723063, 'Disney': 0.2754687640554933, 'toys': 0.2571433497487549, 'Tom Hanks': 0.25586854954717975, 'computer animation': 0.20569575636593754, 'children': 0.1684197159431678, 'witty': 0.13409409827744667, 'animated': 0.12437094618685796, 'funny': 0.1160588017338778, 'family': 0.09118536510172316, 'friendship': 0.08825254466844007, 'comedy': 0.08751341464778947, 'humorous': 0.08259067888269332, 'clever': 0.08029164434290403, 'time travel': 0.07515654418136032, 'adventure': 0.0732247327618749, 'Tim Allen': 0.06542958767652043, 'toy': 0.06469704964954781, 'buddy movie': 0.06396466056401619, 'Cartoon': 0.0571634934911742, 'classic': 0.05148431371496096, 'fun': 0.044812781929927485, 'cgi': 0.04008849830170795, 'fantasy': 0.03893337841870332, 'John Lasseter': 0.03577431461022226, 'imdb top 250': 0.03501466109031044, 'Pixar animation': 0.022073596340099623, 'dolls': 0.021125708856361294}
2 {'Robin Williams

In [193]:
movie_profile[193609]

{'Comedy': 1.0}

# 把上面的代码组合，变为一个整体的处理过程

In [194]:
#整体的处理过程
def create_movie_profile(movie_dataset):
    '''
    使用tfidf，分析提取topn关键词
    :param movie_dataset:
    :return:
    '''
    dataset = movie_dataset["tags"].values

    from gensim.corpora import Dictionary
    # 根据数据集建立词袋，并统计词频，将所有词放入一个词典，使用索引进行获取
    dct = Dictionary(dataset)
    # 根据将每条数据，返回对应的词索引和词频
    corpus = [dct.doc2bow(line) for line in dataset]
    # 训练TF-IDF模型，即计算TF-IDF值
    model = TfidfModel(corpus)

    _movie_profile = []  #本来一个列表，里边放元组，为后面转df做准备
    for i, data in enumerate(movie_dataset.itertuples()):
        mid = data[0]
        title = data[1]
        genres = data[2] #这里是PGC
        vector = model[corpus[i]]
        movie_tags = sorted(vector, key=lambda x: x[1], reverse=True)[:30]  #拿前30个
        topN_tags_weights = dict(map(lambda x: (dct[x[0]], x[1]), movie_tags)) #把词id变为词名字
        # 将类别词的添加进去，并设置权重值为1.0，题材是PGC的
        for g in genres:
            topN_tags_weights[g] = 1.0  #这个for循环把PGC和UGC结合到了一起,原本词在前30个里边，就改为1.0权重，如果没有就添加进去
        topN_tags = [i[0] for i in topN_tags_weights.items()] #取topN_tags_weights中所有的key
        _movie_profile.append((mid, title, topN_tags, topN_tags_weights))

    movie_profile = pd.DataFrame(_movie_profile, columns=["movieId", "title", "profile", "weights"])
    movie_profile.set_index("movieId", inplace=True)
    return movie_profile

In [195]:
movie_profile = create_movie_profile(movie_dataset)

In [196]:
movie_profile.head()

,title,profile,weights
movieId,,,
1,Toy Story (1995),"[Pixar, pixar, animation, Disney, toys, Tom Ha...","{'Pixar': 0.5233593756732888, 'pixar': 0.42629..."
2,Jumanji (1995),"[Robin Williams, time travel, fantasy, board g...","{'Robin Williams': 0.6593901831527829, 'time t..."
3,Grumpier Old Men (1995),"[moldy, Walter Matthau, Jack Lemmon, fishing, ...","{'moldy': 0.42779694952991687, 'Walter Matthau..."
4,Waiting to Exhale (1995),"[chick flick, girl movie, characters, interrac...","{'chick flick': 0.6101947488248344, 'girl movi..."
5,Father of the Bride Part II (1995),"[steve martin, pregnancy, Touching, Diane Keat...","{'steve martin': 0.6280457524844757, 'pregnanc..."


In [247]:
movie_profile.iloc[0].to_dict()

{'title': 'Toy Story (1995)',
 'profile': ['Pixar',
  'pixar',
  'animation',
  'Disney',
  'toys',
  'Tom Hanks',
  'computer animation',
  'children',
  'witty',
  'animated',
  'funny',
  'family',
  'friendship',
  'comedy',
  'humorous',
  'clever',
  'time travel',
  'adventure',
  'Tim Allen',
  'toy',
  'buddy movie',
  'Cartoon',
  'classic',
  'fun',
  'cgi',
  'fantasy',
  'John Lasseter',
  'imdb top 250',
  'Pixar animation',
  'dolls',
  'Adventure',
  'Animation',
  'Children',
  'Comedy',
  'Fantasy'],
 'weights': {'Pixar': 0.5233593756732888,
  'pixar': 0.42629530966659107,
  'animation': 0.36262507978723063,
  'Disney': 0.2754687640554933,
  'toys': 0.2571433497487549,
  'Tom Hanks': 0.25586854954717975,
  'computer animation': 0.20569575636593754,
  'children': 0.1684197159431678,
  'witty': 0.13409409827744667,
  'animated': 0.12437094618685796,
  'funny': 0.1160588017338778,
  'family': 0.09118536510172316,
  'friendship': 0.08825254466844007,
  'comedy': 0.0875134

In [197]:
#为了证明movie_profile中的权重值都是小于1的
a=[]
for d in movie_profile["weights"].values:
    for i in d.values():
        a.append(i)
max(a)

1.0

In [198]:
for mid, weights in movie_profile["weights"].items():
    print(mid,weights)
    break

1 {'Pixar': 0.5233593756732888, 'pixar': 0.42629530966659107, 'animation': 0.36262507978723063, 'Disney': 0.2754687640554933, 'toys': 0.2571433497487549, 'Tom Hanks': 0.25586854954717975, 'computer animation': 0.20569575636593754, 'children': 0.1684197159431678, 'witty': 0.13409409827744667, 'animated': 0.12437094618685796, 'funny': 0.1160588017338778, 'family': 0.09118536510172316, 'friendship': 0.08825254466844007, 'comedy': 0.08751341464778947, 'humorous': 0.08259067888269332, 'clever': 0.08029164434290403, 'time travel': 0.07515654418136032, 'adventure': 0.0732247327618749, 'Tim Allen': 0.06542958767652043, 'toy': 0.06469704964954781, 'buddy movie': 0.06396466056401619, 'Cartoon': 0.0571634934911742, 'classic': 0.05148431371496096, 'fun': 0.044812781929927485, 'cgi': 0.04008849830170795, 'fantasy': 0.03893337841870332, 'John Lasseter': 0.03577431461022226, 'imdb top 250': 0.03501466109031044, 'Pixar animation': 0.022073596340099623, 'dolls': 0.021125708856361294, 'Adventure': 1.0, 

## 开始倒排索引

In [199]:
# 通过标签找到对应的电影，这里就是倒排索引(百度搜索引擎)
def create_inverted_table(movie_profile):
    inverted_table = {}  #初始化倒排索引字典
    for mid, weights in movie_profile["weights"].items():
        for tag, weight in weights.items():#weights是一个字典
            #到inverted_table dict 用tag作为Key去取值 如果取不到就返回[]
            _ = inverted_table.get(tag, []) #字典中是否有某词的倒排索引，没有就返回[]
            _.append((mid, weight))  #对应词在某部电影中的权重
            inverted_table.setdefault(tag, _) #设置对应词的倒排索引
    return inverted_table

#对应格式是{‘action’:[(1,0.90),(2,0.80)]}，每个词作为key，值是列表中嵌套元组，元素第一个元素是电影id
#第二个元素是对应词在该电影中的tiidf值
inverted_table = create_inverted_table(movie_profile)

In [200]:
inverted_table['Pixar']

[(1, 0.5233593756732888),
 (2253, 0.08060677780149383),
 (2294, 0.24843596185748507),
 (2355, 0.8704916133881555),
 (3114, 0.8141301246781383),
 (4886, 0.7952264328049585),
 (6377, 0.725614881830081),
 (8961, 0.7413985205873672),
 (42191, 0.580887311334164),
 (45517, 0.7315483181626741),
 (50872, 0.09093888127866445),
 (68954, 0.6961085551645065),
 (72356, 0.7348256696855984),
 (78499, 0.7354850816934453),
 (83803, 0.4656713968492418),
 (85736, 0.6144140710428143),
 (87876, 0.14683498801702624),
 (88345, 0.38087462463458366),
 (91233, 0.5097635682802009),
 (95167, 0.3938832848770975),
 (95311, 0.5200328699352345),
 (95313, 0.45825854066586),
 (95377, 0.6066015093593168),
 (95654, 0.5097373195324562),
 (95858, 0.5177158984002748),
 (103141, 0.7242727976629832),
 (134853, 0.5226361790662774),
 (135887, 0.09186077359905928),
 (136016, 0.3892980770796846),
 (157296, 0.05976523103337198),
 (187541, 0.3765162919948169)]

## 假如某个用户喜欢action标签，我们会给他推荐什么电影呢？

In [201]:
#比如我们输入pixar，就会得到对应的电影，极其tf-idf值，根据
sorted(inverted_table['action'], key=lambda x: x[1], reverse=True)[0:10]


[(204, 0.946924599881428),
 (109576, 0.944533908046309),
 (4387, 0.713815601385229),
 (1722, 0.5677222712112712),
 (33158, 0.5666358984555815),
 (4565, 0.5389679275916885),
 (3986, 0.5166814606452856),
 (5313, 0.4903148333569994),
 (59947, 0.4134463712512179),
 (1608, 0.4056229785338934)]

In [202]:
ret

,title,genres,tag
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[animated, buddy movie, Cartoon, cgi, comedy, ..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[fantasy, adapted from:book, animals, bad cgi,..."
3,Grumpier Old Men (1995),"[Comedy, Romance]","[moldy, old, Ann Margaret, Burgess Meredith, D..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[characters, girl movie, characters, chick fli..."
5,Father of the Bride Part II (1995),[Comedy],"[steve martin, steve martin, pregnancy, remake..."
...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),"[Action, Animation, Comedy, Fantasy]",NaN
193583,No Game No Life: Zero (2017),"[Animation, Comedy, Fantasy]",NaN
193585,Flint (2017),[Drama],NaN


In [203]:
ret.loc[3285]

title                                     Beach, The (2000)
genres                                   [Adventure, Drama]
tag       [Leonardo DiCaprio, not one boring mistake, Ba...
Name: 3285, dtype: object

# 09-用户画像

### 思考：我们如何对用户进行画像呢？

In [204]:
#步骤
#1 拿到所有电影标签
#2 根据用户看过的电影，从而给用户打上对应标签

In [205]:
## 创建用户画像
watch_record = pd.read_csv("./datasets/ml-latest-small/ratings.csv", usecols=range(2), dtype={"userId":np.int32, "movieId": np.int32})
watch_record = watch_record.groupby("userId").agg(list)

In [206]:
watch_record.head()

,movieId
userId,
1,"[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163,..."
2,"[318, 333, 1704, 3578, 6874, 8798, 46970, 4851..."
3,"[31, 527, 647, 688, 720, 849, 914, 1093, 1124,..."
4,"[21, 32, 45, 47, 52, 58, 106, 125, 126, 162, 1..."
5,"[1, 21, 34, 36, 39, 50, 58, 110, 150, 153, 232..."


In [207]:
movie_profile.head()

,title,profile,weights
movieId,,,
1,Toy Story (1995),"[Pixar, pixar, animation, Disney, toys, Tom Ha...","{'Pixar': 0.5233593756732888, 'pixar': 0.42629..."
2,Jumanji (1995),"[Robin Williams, time travel, fantasy, board g...","{'Robin Williams': 0.6593901831527829, 'time t..."
3,Grumpier Old Men (1995),"[moldy, Walter Matthau, Jack Lemmon, fishing, ...","{'moldy': 0.42779694952991687, 'Walter Matthau..."
4,Waiting to Exhale (1995),"[chick flick, girl movie, characters, interrac...","{'chick flick': 0.6101947488248344, 'girl movi..."
5,Father of the Bride Part II (1995),"[steve martin, pregnancy, Touching, Diane Keat...","{'steve martin': 0.6280457524844757, 'pregnanc..."


In [208]:
user_profile ={}
for uid,mids in watch_record.itertuples():
    #从电影的关键词权重数据中 取出当前用户看过的所有电影，movie_profile是上面已经计算好的电影画像
    record_moive_profile = movie_profile.loc[list(mids)]
    print(uid)
    print(record_moive_profile)
    break

1
                                  title  \
movieId                                   
1                      Toy Story (1995)   
3               Grumpier Old Men (1995)   
6                           Heat (1995)   
47          Seven (a.k.a. Se7en) (1995)   
50           Usual Suspects, The (1995)   
...                                 ...   
3744                       Shaft (2000)   
3793                       X-Men (2000)   
3809             What About Bob? (1991)   
4006     Transformers: The Movie (1986)   
5060       M*A*S*H (a.k.a. MASH) (1970)   

                                                   profile  \
movieId                                                      
1        [Pixar, pixar, animation, Disney, toys, Tom Ha...   
3        [moldy, Walter Matthau, Jack Lemmon, fishing, ...   
6        [Al Pacino, Robert De Niro, realistic action, ...   
47       [twist ending, serial killer, Kevin Spacey, po...   
50       [twist ending, Kevin Spacey, complicated, susp...   
... 

In [209]:
record_moive_profile = movie_profile.loc[[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163]]

In [210]:
#每一个用户都有一个这样的record_moive_profile
record_moive_profile

,title,profile,weights
movieId,,,
1,Toy Story (1995),"[Pixar, pixar, animation, Disney, toys, Tom Ha...","{'Pixar': 0.5233593756732888, 'pixar': 0.42629..."
3,Grumpier Old Men (1995),"[moldy, Walter Matthau, Jack Lemmon, fishing, ...","{'moldy': 0.42779694952991687, 'Walter Matthau..."
6,Heat (1995),"[Al Pacino, Robert De Niro, realistic action, ...","{'Al Pacino': 0.5327695696707754, 'Robert De N..."
47,Seven (a.k.a. Se7en) (1995),"[twist ending, serial killer, Kevin Spacey, po...","{'twist ending': 0.361418524598396, 'serial ki..."
50,"Usual Suspects, The (1995)","[twist ending, Kevin Spacey, complicated, susp...","{'twist ending': 0.5436045088595716, 'Kevin Sp..."
70,From Dusk Till Dawn (1996),"[Quentin Tarantino, vampires, cult classic, Ge...","{'Quentin Tarantino': 0.5889711207330415, 'vam..."
101,Bottle Rocket (1996),"[Wes Anderson, off-beat comedy, quirky, NOTHIN...","{'Wes Anderson': 0.8459254366520916, 'off-beat..."
110,Braveheart (1995),"[mel gibson, historical, Medieval, Mel Gibson,...","{'mel gibson': 0.4004865422292125, 'historical..."
151,Rob Roy (1995),"[Scotland, sword fight, 18th century, highland...","{'Scotland': 0.45272353419972067, 'sword fight..."


In [211]:
record_moive_profile['profile'].values

array([list(['Pixar', 'pixar', 'animation', 'Disney', 'toys', 'Tom Hanks', 'computer animation', 'children', 'witty', 'animated', 'funny', 'family', 'friendship', 'comedy', 'humorous', 'clever', 'time travel', 'adventure', 'Tim Allen', 'toy', 'buddy movie', 'Cartoon', 'classic', 'fun', 'cgi', 'fantasy', 'John Lasseter', 'imdb top 250', 'Pixar animation', 'dolls', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']),
       list(['moldy', 'Walter Matthau', 'Jack Lemmon', 'fishing', 'old', 'Ann Margaret', 'comedinha de velhinhos engraÃ§ada', 'comedinha de velhinhos engraÃƒÂ§ada', 'grun running', 'old people that is actually funny', 'Sophia Loren', 'Burgess Meredith', 'Howard Deutch', 'Daryl Hannah', 'old man', 'sequel fever', 'Funniest Movies', 'good soundtrack', 'best friend', 'NO_FA_GANES', 'duringcreditsstinger', 'sequel', 'comedy', 'CLV', 'Romance', 'Comedy']),
       list(['Al Pacino', 'Robert De Niro', 'realistic action', 'Michael Mann', 'Val Kilmer', 'great acting', 'bank r

In [248]:
[i for j in record_moive_profile['profile'].values for i in j]  #二维变一维

['Pixar',
 'pixar',
 'animation',
 'Disney',
 'toys',
 'Tom Hanks',
 'computer animation',
 'children',
 'witty',
 'animated',
 'funny',
 'family',
 'friendship',
 'comedy',
 'humorous',
 'clever',
 'time travel',
 'adventure',
 'Tim Allen',
 'toy',
 'buddy movie',
 'Cartoon',
 'classic',
 'fun',
 'cgi',
 'fantasy',
 'John Lasseter',
 'imdb top 250',
 'Pixar animation',
 'dolls',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'moldy',
 'Walter Matthau',
 'Jack Lemmon',
 'fishing',
 'old',
 'Ann Margaret',
 'comedinha de velhinhos engraÃ§ada',
 'comedinha de velhinhos engraÃƒÂ§ada',
 'grun running',
 'old people that is actually funny',
 'Sophia Loren',
 'Burgess Meredith',
 'Howard Deutch',
 'Daryl Hannah',
 'old man',
 'sequel fever',
 'Funniest Movies',
 'good soundtrack',
 'best friend',
 'NO_FA_GANES',
 'duringcreditsstinger',
 'sequel',
 'comedy',
 'CLV',
 'Romance',
 'Comedy',
 'Al Pacino',
 'Robert De Niro',
 'realistic action',
 'Michael Mann',
 'Val Kilmer',


In [249]:
len([i for j in record_moive_profile['profile'].values for i in j])

345

In [212]:
from functools import reduce
import collections  #和容器功能类似，里边有一些特殊的容器
#类似于二维变一维
temp = reduce(lambda x,y : list(x)+list(y),record_moive_profile['profile'].values)
#选取上面11部电影中唯一的标签,把这些标签加起来，这里还没有去重,可以看Comedy
temp 

['Pixar',
 'pixar',
 'animation',
 'Disney',
 'toys',
 'Tom Hanks',
 'computer animation',
 'children',
 'witty',
 'animated',
 'funny',
 'family',
 'friendship',
 'comedy',
 'humorous',
 'clever',
 'time travel',
 'adventure',
 'Tim Allen',
 'toy',
 'buddy movie',
 'Cartoon',
 'classic',
 'fun',
 'cgi',
 'fantasy',
 'John Lasseter',
 'imdb top 250',
 'Pixar animation',
 'dolls',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'moldy',
 'Walter Matthau',
 'Jack Lemmon',
 'fishing',
 'old',
 'Ann Margaret',
 'comedinha de velhinhos engraÃ§ada',
 'comedinha de velhinhos engraÃƒÂ§ada',
 'grun running',
 'old people that is actually funny',
 'Sophia Loren',
 'Burgess Meredith',
 'Howard Deutch',
 'Daryl Hannah',
 'old man',
 'sequel fever',
 'Funniest Movies',
 'good soundtrack',
 'best friend',
 'NO_FA_GANES',
 'duringcreditsstinger',
 'sequel',
 'comedy',
 'CLV',
 'Romance',
 'Comedy',
 'Al Pacino',
 'Robert De Niro',
 'realistic action',
 'Michael Mann',
 'Val Kilmer',


In [213]:
len(temp)

345

In [214]:
#对每一个标签出现的次数进行统计
# count对象像字典，key是标签，value是出现的次数
counter = collections.Counter(temp)

In [215]:
counter

Counter({'Comedy': 5,
         'Action': 5,
         'Romance': 4,
         'Thriller': 4,
         'funny': 3,
         'crime': 3,
         'Crime': 3,
         'twist ending': 3,
         'thriller': 3,
         'War': 3,
         'comedy': 2,
         'clever': 2,
         'classic': 2,
         'imdb top 250': 2,
         'Adventure': 2,
         'great acting': 2,
         'suspense': 2,
         'atmospheric': 2,
         'long': 2,
         'police': 2,
         'Kevin Spacey': 2,
         'great ending': 2,
         'twists & turns': 2,
         'mystery': 2,
         'violent': 2,
         'horror': 2,
         'Mystery': 2,
         'organized crime': 2,
         'conspiracy': 2,
         'Robert Rodriguez': 2,
         'Salma Hayek': 2,
         'mexico': 2,
         'violence': 2,
         'Nudity (Topless - Brief)': 2,
         'Danny Trejo': 2,
         'war': 2,
         'Scotland': 2,
         'sword fight': 2,
         'action': 2,
         'Drama': 2,
         'Weste

In [216]:
len(counter)

288

In [217]:
type(counter)  #是一个Counter对象

collections.Counter

In [218]:
interest_words = counter.most_common(30)  #选择词频较高的前30个词，代表用户感兴趣
interest_words

[('Comedy', 5),
 ('Action', 5),
 ('Romance', 4),
 ('Thriller', 4),
 ('funny', 3),
 ('crime', 3),
 ('Crime', 3),
 ('twist ending', 3),
 ('thriller', 3),
 ('War', 3),
 ('comedy', 2),
 ('clever', 2),
 ('classic', 2),
 ('imdb top 250', 2),
 ('Adventure', 2),
 ('great acting', 2),
 ('suspense', 2),
 ('atmospheric', 2),
 ('long', 2),
 ('police', 2),
 ('Kevin Spacey', 2),
 ('great ending', 2),
 ('twists & turns', 2),
 ('mystery', 2),
 ('violent', 2),
 ('horror', 2),
 ('Mystery', 2),
 ('organized crime', 2),
 ('conspiracy', 2),
 ('Robert Rodriguez', 2)]

In [219]:
maxcount = interest_words[0][1]
maxcount    #用感兴趣的词出现的词频，除以最大词频来得出每个兴趣词的分值

5

## 用户画像整体流程

In [ ]:

def create_user_profile():
    watch_record = pd.read_csv("./datasets/ml-latest-small/ratings.csv", usecols=range(2), dtype={"userId":np.int32, "movieId": np.int32})

    watch_record = watch_record.groupby("userId").agg(list)
    # print(watch_record)

    user_profile = {}
    for uid, mids in watch_record.itertuples():
        #movie_profile行索引是movie id，profile列记录了每一步电影有哪些标签
        record_movie_prifole = movie_profile.loc[list(mids)]
        #二维数组转一维数组，把用户看过的电影的标签都放到一个一维数组中
        counter = collections.Counter(reduce(lambda x, y: list(x)+list(y), record_movie_prifole["profile"].values))
        # 兴趣词,前20
        interest_words = counter.most_common(20)
        maxcount = interest_words[0][1]
        #把感兴趣的词变为0-1直接的分值
        interest_words = [(w,round(c/maxcount, 4)) for w,c in interest_words]
        user_profile[uid] = interest_words

    return user_profile

user_profile = create_user_profile()

In [221]:
#用户画像结果输出
for k,v in user_profile.items():
    print(k,v)
    if k>2:
        break

1 [('Action', 1.0), ('Adventure', 0.9556), ('Comedy', 0.9222), ('Drama', 0.7556), ('Thriller', 0.6222), ('classic', 0.5667), ('Fantasy', 0.5222), ('Crime', 0.5), ('Children', 0.4667), ('Sci-Fi', 0.4444), ('action', 0.4), ('comedy', 0.3556), ('adventure', 0.3444), ('fantasy', 0.3222), ('Animation', 0.3222), ('sci-fi', 0.3222), ('based on a book', 0.3), ('Romance', 0.2889), ('Disney', 0.2778), ('funny', 0.2778)]
2 [('Drama', 1.0), ('Action', 0.6471), ('Crime', 0.5882), ('violence', 0.5882), ('Thriller', 0.5882), ('action', 0.4706), ('imdb top 250', 0.4118), ('Comedy', 0.4118), ('visually appealing', 0.4118), ('funny', 0.3529), ('twist ending', 0.2941), ('great acting', 0.2941), ('atmospheric', 0.2941), ('revenge', 0.2941), ('psychology', 0.2941), ('violent', 0.2941), ('dark', 0.2941), ('Leonardo DiCaprio', 0.2941), ('thriller', 0.2941), ('thought-provoking', 0.2353)]
3 [('Drama', 1.0), ('Sci-Fi', 0.9375), ('Action', 0.875), ('Adventure', 0.6875), ('Comedy', 0.5625), ('Horror', 0.5), ('cl

# 10 有了用户画像，物品画像，如何实现推荐top-10的电影呢？

In [222]:
#产生推荐结果---任何一个用户感兴趣的前10部电影
for uid, interest_words in user_profile.items():
    result_table = {}#要推荐的结果，用一个字典保存
    for interest_word, interest_weight in interest_words:
        #通过inverted_table倒排索引表得到对应词，相关那些电影
        related_movies = inverted_table[interest_word]
        for mid, related_weight in related_movies:
            # mid 电影id related_weight
            _ = result_table.get(mid, []) # 字典中是否有某电影的推荐结果，没有就返回[]
#             _.append(interest_weight)    # 只考虑用户的兴趣程度
            # _.append(related_weight)    # 只考虑兴趣词与电影的关联程度
            _.append(interest_weight*related_weight)    # 二者都考虑，
            result_table.setdefault(mid, _)
    # print(result_table)
    print('-'*100)
    #因为返回的是键值对，键是电影id，值是感兴趣程度的list，所以sum求和
    rs_result = map(lambda x: (x[0], sum(x[1])), result_table.items())
    rs_result = sorted(rs_result, key=lambda x:x[1], reverse=True)[:10]  #求和后的结果降序排序
    print(uid)
    print(rs_result) #电影id，及该用户对这个电影的感兴趣程度，总计10个


----------------------------------------------------------------------------------------------------
1
[(117646, 4.7778), (81132, 4.7556), (71999, 4.5889), (4956, 4.5445), (164226, 4.4666), (4719, 4.4111), (26340, 4.320797362051149), (546, 4.3111), (31367, 4.2889), (85261, 4.2814095503224285)]
----------------------------------------------------------------------------------------------------
2
[(145, 3.3677020232992003), (7235, 3.3206600269158075), (5628, 3.3054219829044027), (1432, 3.2821078644895727), (7007, 3.2674694629722296), (20, 3.2353), (4719, 3.2353), (5027, 3.2353), (81132, 3.2353), (2959, 3.224503665653853)]
----------------------------------------------------------------------------------------------------
3
[(71999, 4.25), (48774, 4.130975708110577), (81132, 4.0625), (91500, 4.045859646978439), (117529, 4.013791940473096), (8361, 3.974914351004604), (187031, 3.968179667080631), (58025, 3.943875608037196), (27618, 3.9375), (4956, 3.905648707284492)]
-----------------------

In [223]:
movie_dataset.loc[117646]

title                 Dragonheart 2: A New Beginning (2000)
genres    [Action, Adventure, Comedy, Drama, Fantasy, Th...
tags      [Action, Adventure, Comedy, Drama, Fantasy, Th...
Name: 117646, dtype: object

In [224]:
user_profile[1]

[('Action', 1.0),
 ('Adventure', 0.9556),
 ('Comedy', 0.9222),
 ('Drama', 0.7556),
 ('Thriller', 0.6222),
 ('classic', 0.5667),
 ('Fantasy', 0.5222),
 ('Crime', 0.5),
 ('Children', 0.4667),
 ('Sci-Fi', 0.4444),
 ('action', 0.4),
 ('comedy', 0.3556),
 ('adventure', 0.3444),
 ('fantasy', 0.3222),
 ('Animation', 0.3222),
 ('sci-fi', 0.3222),
 ('based on a book', 0.3),
 ('Romance', 0.2889),
 ('Disney', 0.2778),
 ('funny', 0.2778)]

# 11 物品冷启动（电影画像）处理(word2vec与Doc2Vec)

In [225]:
#下面是另外一种方法做物品画像方法，分别是 Word2Vec和Doc2Vec

In [226]:
movie_profile["profile"]

movieId
1         [Pixar, pixar, animation, Disney, toys, Tom Ha...
2         [Robin Williams, time travel, fantasy, board g...
3         [moldy, Walter Matthau, Jack Lemmon, fishing, ...
4         [chick flick, girl movie, characters, interrac...
5         [steve martin, pregnancy, Touching, Diane Keat...
                                ...                        
193581                 [Animation, Fantasy, Action, Comedy]
193583                         [Animation, Fantasy, Comedy]
193585                                              [Drama]
193587                                  [Animation, Action]
193609                                             [Comedy]
Name: profile, Length: 9742, dtype: object

In [227]:
sentences = list(movie_profile["profile"].values)
sentences

[['Pixar',
  'pixar',
  'animation',
  'Disney',
  'toys',
  'Tom Hanks',
  'computer animation',
  'children',
  'witty',
  'animated',
  'funny',
  'family',
  'friendship',
  'comedy',
  'humorous',
  'clever',
  'time travel',
  'adventure',
  'Tim Allen',
  'toy',
  'buddy movie',
  'Cartoon',
  'classic',
  'fun',
  'cgi',
  'fantasy',
  'John Lasseter',
  'imdb top 250',
  'Pixar animation',
  'dolls',
  'Adventure',
  'Animation',
  'Children',
  'Comedy',
  'Fantasy'],
 ['Robin Williams',
  'time travel',
  'fantasy',
  'board game',
  'kid flick',
  'bad cgi',
  'animals',
  'magic board game',
  'scary',
  'Kirsten Dunst',
  'monkey',
  'not for kids',
  'Chris Van Allsburg',
  'childhood recaptured',
  'Filmed in BC',
  'Joe Johnston',
  'Children',
  'game',
  'time',
  'jungle',
  'Adaptation of Book',
  'see also:Zathura',
  'adapted from:book',
  'Lebbat',
  'thrill',
  'For children',
  'herds of CGI animals',
  'giant insect',
  'new home',
  'Saturn Award (Best Suppo

In [228]:
import gensim
sentences = list(movie_profile["profile"].values)
#window 代表一次看3个词  iter=20 epoch的次数,min_count总频率小于1，就忽略掉
model = gensim.models.Word2Vec(sentences, window=3, min_count=1)

In [229]:
help(gensim.models.Word2Vec)

Help on class Word2Vec in module gensim.models.word2vec:

class Word2Vec(gensim.utils.SaveLoad)
 |  Word2Vec(sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), comment=None, max_final_vocab=None, shrink_windows=True)
 |
 |  Method resolution order:
 |      Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, co

In [230]:
#下面我们来看下和action关键词相似的词向量，
# 由于我们这些不是一篇文章(电影），是一些独立的词，所以效果不是很好，对于一篇篇使用word2vec非常适合的
model.wv.most_similar(positive=['action'], topn=10)

[('sequel', 0.9937356114387512),
 ('predictable', 0.9913654923439026),
 ('70mm', 0.9903773665428162),
 ('DVD-RAM', 0.9885210394859314),
 ('classic', 0.9861803650856018),
 ('cult film', 0.9860755801200867),
 ('friendship', 0.985921323299408),
 ('dvd', 0.9848561882972717),
 ('mine', 0.9848126769065857),
 ('Bob*ola', 0.9846264123916626)]

In [231]:
if 'action' in model.wv:
    word_vector = model.wv['action']

    # 打印这个词的向量
    print(len(word_vector))

100


## Doc2Vec

In [232]:
movie_profile["profile"]

movieId
1         [Pixar, pixar, animation, Disney, toys, Tom Ha...
2         [Robin Williams, time travel, fantasy, board g...
3         [moldy, Walter Matthau, Jack Lemmon, fishing, ...
4         [chick flick, girl movie, characters, interrac...
5         [steve martin, pregnancy, Touching, Diane Keat...
                                ...                        
193581                 [Animation, Fantasy, Action, Comedy]
193583                         [Animation, Fantasy, Comedy]
193585                                              [Drama]
193587                                  [Animation, Action]
193609                                             [Comedy]
Name: profile, Length: 9742, dtype: object

In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
# https://www.jianshu.com/p/048bff9b0f65
documents = [TaggedDocument(words, [movie_id]) for movie_id, words in movie_profile["profile"].items()] 


In [234]:
documents[0:5] #前面是电影的words，后面是id，因为Doc2Vec的要求

[TaggedDocument(words=['Pixar', 'pixar', 'animation', 'Disney', 'toys', 'Tom Hanks', 'computer animation', 'children', 'witty', 'animated', 'funny', 'family', 'friendship', 'comedy', 'humorous', 'clever', 'time travel', 'adventure', 'Tim Allen', 'toy', 'buddy movie', 'Cartoon', 'classic', 'fun', 'cgi', 'fantasy', 'John Lasseter', 'imdb top 250', 'Pixar animation', 'dolls', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], tags=[1]),
 TaggedDocument(words=['Robin Williams', 'time travel', 'fantasy', 'board game', 'kid flick', 'bad cgi', 'animals', 'magic board game', 'scary', 'Kirsten Dunst', 'monkey', 'not for kids', 'Chris Van Allsburg', 'childhood recaptured', 'Filmed in BC', 'Joe Johnston', 'Children', 'game', 'time', 'jungle', 'Adaptation of Book', 'see also:Zathura', 'adapted from:book', 'Lebbat', 'thrill', 'For children', 'herds of CGI animals', 'giant insect', 'new home', 'Saturn Award (Best Supporting Actress)', 'Adventure', 'Fantasy'], tags=[2]),
 TaggedDocument(word

In [235]:
# 训练模型并保存 Doc2Vec 通过向量来表示一篇文档  一篇文档就对应一个电影
#向量的相似度 代表了电影额相似程度，vector_size代表最终生成的向量是100维的
# 词频少于min_count次数的单词会被丢弃掉, 默认值为5
model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, workers=4, epochs=20)

In [236]:
#取出索引是6的电影
words = movie_profile["profile"].loc[5]
words

['steve martin',
 'pregnancy',
 'Touching',
 'Diane Keaton',
 'family',
 'worst movies ever',
 'wedding',
 'watched under duress',
 'contraception',
 'gynecologist',
 "it thought it was funny but it wasn't",
 'sequel fever',
 'confidence',
 'childhood classics',
 'midlife crisis',
 'Steve Martin',
 'aging',
 'baby',
 'daughter',
 'remake',
 'parent child relationship',
 'sequel',
 'Comedy',
 'Fantasy',
 'CLV']

In [237]:
movie_profile.loc[5]

title                     Father of the Bride Part II (1995)
profile    [steve martin, pregnancy, Touching, Diane Keat...
weights    {'steve martin': 0.6280457524844757, 'pregnanc...
Name: 5, dtype: object

In [238]:
#模型训练好了，拿到第5个电影id的文章向量
inferred_vector = model.infer_vector(words)
inferred_vector.shape

(100,)

In [239]:
inferred_vector

array([ 0.06748549,  0.06121351,  0.16520342, -0.23326313, -0.15120769,
       -0.19338672, -0.144952  , -0.08509969,  0.22811474, -0.03156864,
       -0.08757883, -0.02747009, -0.19221395, -0.16854662,  0.06335487,
       -0.01213049, -0.09694947, -0.25971133,  0.16967419,  0.05825761,
        0.06080598, -0.02796307, -0.15053076, -0.05709666,  0.11489566,
        0.14312273, -0.122285  , -0.2556343 ,  0.02752171,  0.00821436,
       -0.05607727,  0.1401564 , -0.02741837, -0.04028743,  0.14952478,
       -0.14136025, -0.13081859,  0.00267153,  0.11644425, -0.12496104,
       -0.07445663, -0.02960306,  0.03703569, -0.11839283, -0.11860748,
       -0.22192265,  0.07765344,  0.10267257, -0.14264387,  0.16110346,
       -0.0169001 ,  0.2133382 ,  0.05979455, -0.01661093,  0.18091927,
        0.04425272,  0.06765097,  0.21164432,  0.02871539,  0.22366904,
       -0.10496014,  0.03418216, -0.26223883,  0.07049607,  0.11322007,
       -0.11411553,  0.11566058, -0.19331948,  0.11284404,  0.03

In [240]:
sims = model.dv.most_similar([inferred_vector], topn=10)  #找出最相似的10部电影

In [241]:
sims

[(6944, 0.9609112739562988),
 (4447, 0.9586201310157776),
 (32289, 0.9548001885414124),
 (4476, 0.9526134133338928),
 (333, 0.9477721452713013),
 (64032, 0.9466479420661926),
 (4700, 0.9457805752754211),
 (1307, 0.9419568181037903),
 (3087, 0.9410703182220459),
 (5620, 0.9407297372817993)]

In [242]:
#因为最相似的取一个
movie_profile.loc[6338]

title                                  Daddy Day Care (2003)
profile    [Eddie Murphy, Steve Carr, father-son relation...
weights    {'Eddie Murphy': 0.4905951071756971, 'Steve Ca...
Name: 6338, dtype: object

In [243]:
movie_profile['profile'].loc[91266]

['retelling of a fairy tale',
 'Cinderella',
 'has been',
 'masked ball',
 'modern fairy tale',
 'high school',
 'cinderella',
 'maid',
 'pop star',
 'dancer',
 'celebrity',
 'teens',
 'orphan',
 'dance',
 'dancing',
 'fairy tale',
 'teenager',
 'musical',
 'Musical',
 'music',
 'Children',
 'Romance',
 'Comedy']

Doc2Vec是一种基于Word2Vec模型的扩展，用于学习文档（或句子）的向量表示。它是由Tomas Mikolov等人在2014年提出的。

训练Doc2Vec模型的基本思想是将文档（或句子）作为一个整体来学习其向量表示，而不是仅仅考虑单词的向量表示。Doc2Vec模型有两个变种：Distributed Memory version of Paragraph Vectors (PV-DM)和Distributed Bag of Words version of Paragraph Vectors (PV-DBOW)。

Distributed Memory version of Paragraph Vectors (PV-DM)：

对于每个文档，为它分配一个唯一的标识符（paragraph ID）。
对于每个文档，将其标识符与每个单词的向量表示拼接在一起，形成一个输入向量。
通过类似于CBOW（Continuous Bag of Words）的方法，将输入向量作为输入预测文档中的目标单词。
在预测过程中，不仅更新目标单词的向量表示，还更新文档向量的表示，使其能够捕捉到文档的整体语义信息。
迭代训练这个模型，通过不断调整文档向量和单词向量，使得模型能够更好地预测目标单词。

Distributed Bag of Words version of Paragraph Vectors (PV-DBOW)：

对于每个文档，为它分配一个唯一的标识符（paragraph ID）。
通过随机采样或者滑动窗口的方式，在文档中选择一个单词作为目标单词。
直接使用文档的标识符作为输入，预测目标单词。
在预测过程中，只更新目标单词的向量表示，不更新文档向量的表示。
迭代训练这个模型，通过不断调整单词向量，使得模型能够更好地预测目标单词。
无论是PV-DM还是PV-DBOW，都可以使用梯度下降等优化算法来训练模型。在训练过程中，通过最小化预测目标单词与真实目标单词之间的差距（通常使用负对数似然函数），不断更新文档向量和单词向量的表示。

一旦训练完成，可以使用得到的文档向量来表示文档，这些向量可以用于计算文档之间的相似度、聚类等任务。

在Distributed Memory version of Paragraph Vectors (PV-DM) 中，输入和输出如下所示：

输入：

文档集合：包含多个文档（或句子）的语料库。
单词向量：对语料库中的每个单词，都有一个事先训练好的单词向量表示。
输出：

文档向量：对于每个文档（或句子），学习到一个固定长度的向量表示，用于表示该文档的语义信息。
具体来说，对于每个文档（或句子），PV-DM 模型的输入和输出如下所示：

输入：

上下文单词：从文档中选择一个单词作为目标单词，然后根据预定义的窗口大小，获取目标单词周围的上下文单词作为输入。例如，如果窗口大小为2，则选择目标单词的前两个单词和后两个单词作为上下文单词。

文档标识符（paragraph ID）：为每个文档分配一个唯一的标识符。

输出：

目标单词：PV-DM 模型的任务是根据上下文单词和文档标识符，预测目标单词。这是一个多分类问题，根据词汇表的大小，使用 softmax 或类似的激活函数进行预测。

文档向量：在预测目标单词的同时，模型也更新文档的向量表示。文档向量的维度与单词向量的维度相同，通常是固定的。

通过不断迭代训练 PV-DM 模型，调整文档向量和单词向量的表示，以最小化预测目标单词与真实目标单词之间的差距。最终，得到的文档向量可以用于表示文档，进行文档间的相似度计算、聚类等任务。

在Distributed Bag of Words version of Paragraph Vectors (PV-DBOW) 中，输入和输出如下所示：

输入：

文档集合：包含多个文档（或句子）的语料库。
输出：

文档向量：对于每个文档（或句子），学习到一个固定长度的向量表示，用于表示该文档的语义信息。
具体来说，对于每个文档（或句子），PV-DBOW 模型的输入和输出如下所示：

输入：

文档标识符（paragraph ID）：为每个文档分配一个唯一的标识符。
输出：

目标单词：PV-DBOW 模型的任务是根据文档标识符，预测目标单词。这是一个多分类问题，根据词汇表的大小，使用 softmax 或类似的激活函数进行预测。

文档向量：PV-DBOW 模型不使用上下文单词作为输入，而是仅仅通过文档标识符来预测目标单词。在预测目标单词的同时，模型也不会更新文档向量的表示。

通过不断迭代训练 PV-DBOW 模型，调整目标单词的向量表示，以最小化预测目标单词与真实目标单词之间的差距。最终，得到的文档向量可以用于表示文档，进行文档间的相似度计算、聚类等任务。

需要注意的是，PV-DBOW 模型与 PV-DM 模型相比，不考虑上下文单词信息，只使用文档标识符进行预测。这使得 PV-DBOW 模型更适合于处理长文档或句子，或者在数据稀疏的情况下。